In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount = True)

Mounted at /content/drive/


In [ ]:
# !pip install jiwer
import os
import glob
import pandas as pd
import os
# from jiwer import wer,cer
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', 0)

# TF-IDF Analysis

In [ ]:
import re
string = "তিন গাছ তিন গাছ ছয় গাছ দেবে। "
re.findall(r"[\u0980-\u09FF\u200d]{1,}", string)

['তিন', 'গাছ', 'তিন', 'গাছ', 'ছয়', 'গাছ', 'দেবে']

In [ ]:
test_df = pd.read_excel('/content/drive/MyDrive/local_speech/others/IUT_bhasha_bichitra/orig_iut_folds/fix_dataset/final/test/test.xlsx')
test_df

,file_name,External_ID,district,transcriptions,Split,annotator
0,test_barishal (1).wav,rec_21_audio_36.wav,barishal,"তিন গাছ তিন গাছ ছয় গাছ দেবে। তুই আয় এই জাগা <> মুই বাত খাই খাওয়া অইয়া গ্যাছে। হাজা-গাজায় জম্মের পডু ব্যাডা, এট্টু মেহেদী, এট্টু লেবেস্টিক, এট্টু আইবুরু দেলে হেয়ার আর পিছ ছাড়ে না",Test,nahar2709marzan@gmail.com
1,test_barishal (10).wav,rec_21_audio_45.wav,barishal,"গরম লাগে <> কী আছে? কই দেহি। ও পড়বে হোনেন না? ও দেইক্কা মনে অয় ই করছে। না এতে গরম লাগে তো, হাত <> দেয় নাই, ইদের সময় কিনছি না, কাইলগো ওই পোলায় হেই রাত্তিরে বাইর কইররা পড়াই দেছে <>",Test,nahar2709marzan@gmail.com
2,test_barishal (100).wav,rec_24_audio_10.wav,barishal,"চেড়তে চেড়তে আগা বাইরাইয়া যায়। গত বছছর যহন এতো গাছ এই যে লেবারি করলাম এই গাছ, বড় বড় গাছ, যত বড় গাছ অউক <> হেই গাছে ওটছো?",Test,nahar2709marzan@gmail.com
3,test_barishal (101).wav,rec_24_audio_11.wav,barishal,"ওরে আর, ওয়া তো যেকালে পারছো হেইকালে গাছে ওটছো <> কাম করা ভালো, এহন এই যে কিনছি এয়ারও কারন আছে, হ চলান-মলান লইয়া",Test,nahar2709marzan@gmail.com
4,test_barishal (102).wav,rec_24_audio_12.wav,barishal,"মোর এট্টা ইসাব, কী? চা আর খামু না অনেক খাইছি। বোজ্জো? গাছের কাম যত দিকদা বাইরাইয়া অন্য কিছু করার চিন্তা আছে। হুম? এই জগতেইত্তা বাইরাইয়া অন্য কিছু করার চিন্তা আছে। হ, হ, অতোডু সময় কইরা-ঠইরা কিছু জোগাইতারলে হেইপর আর, এই জেন্যে এই গাছ কিনছি এহন ধরেন বেইচ্চা-বুইচ্চা, বেইচ্চা-বুইচ্চা জোগার কইরা জোগাড় করতে, আর ওই টাহা লাব, ওই লাবের টাহাইদ্দা",Test,nahar2709marzan@gmail.com
...,...,...,...,...,...,...
1677,test_tangail (95).wav,rec_39_audio_97.wav,tangail,"সো ছবিটা আসলে দেখলে মানে ভাল্লাগবো অনেক। তারপরে আমি যেই মুভিডা শেয়ার করমু, সেইডা হইছে সাভারিয়া। এইডা দুই হাজার সাতের ছবি। এই গানগুলা আমি শুনছিলাম কিন্তু এর এই ছবিডা আমার কহোনোই আর কি দেহা",Test,zubaer71.bd@gmail.com
1678,test_tangail (96).wav,rec_39_audio_98.wav,tangail,"পরে ইদানীং আমার কেমনে জানি মনে অইলো যে এই ছবিডা দ্যাহি। এই ছবিডা দ্যাহা অয় নাই। পরে এই ছবিডা মানে ডাউনলোড কইরা দেখতে দ্যাহা শুরু করলাম। দুই হাজার সাত সালের ছবি এইডা অইছে এই দেবদাসের যে ডিরেক্টর আছে, পদ্মাবতের যে ডিরেক্টর আছে, এ",Test,zubaer71.bd@gmail.com
1679,test_tangail (97).wav,rec_39_audio_99.wav,tangail,"বলিউডের অন্যতম এক প্রভাবশালী ডিরেক্টর সঞ্জয় লীলা বানসালি, তার তৈরি। আর রনবীর কাপুর তহন, রনবীর কাপুরের ওইডা প্রথম ছবি। আর ছবিতে সালমান খান, সোনাম কাপুর, রাণী মুখার্জিও আছে। মেইন ক্যারেক্টার আছিলো এই চারজনেই।",Test,zubaer71.bd@gmail.com
1680,test_tangail (98).wav,rec_39_audio_100.wav,tangail,"<> মুটামুটি এই চারজন বাদে আর কেউ নেই। আর এই ছবিতে আরেকটা বিষয় অইছে, এই ছবিতে আসলে মানুষ তেমন নাই ই। খুব অল্প মানুষ নিয়ে এই ছবিডা মানে মেক করা অইছে। আপনি পুরা ছবি দেখলেও মানে হাতেগোণা মানে দশজনেরও মনে অয় কম মানুষ নিয়া এই ছবিডা বানানো।",Test,zubaer71.bd@gmail.com


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re


bangla_stopwords = [
    'অতএব', 'অথচ', 'অথবা', 'অনুযায়ী', 'অনেক', 'অনেকে', 'অনেকেই', 'অন্তত', 'অন্য', 'অবধি',
    'অবশ্য', 'অর্থাত', 'আই', 'আগামী', 'আগে', 'আগেই', 'আছে', 'আজ', 'আদ্যভাগে', 'আপনার',
    'আপনি', 'আবার', 'আমরা', 'আমাকে', 'আমাদের', 'আমার', 'আমি', 'আর', 'আরও', 'ই', 'ইত্যাদি',
    'ইহা', 'উচিত', 'উত্তর', 'উনি', 'উপর', 'উপরে', 'এ', 'এঁদের', 'এঁরা', 'এই', 'একই', 'একটি',
    'একবার', 'একে', 'এক্', 'এখন', 'এখনও', 'এখানে', 'এখানেই', 'এটা', 'এটাই', 'এটি', 'এত',
    'এতটাই', 'এতে', 'এদের', 'এব', 'এবং', 'এবার', 'এমন', 'এমনকী', 'এমনি', 'এর', 'এরা', 'এল',
    'এস', 'এতে', 'ঐ', 'ও', 'ওঁদের', 'ওঁর', 'ওঁরা', 'ওই', 'ওকে', 'ওখানে', 'ওদের', 'ওর', 'ওরা',
    'কখনও', 'কত', 'কবে', 'কমনে', 'কয়েক', 'কয়েকটি', 'করছে', 'করছেন', 'করতে', 'করবে', 'করবেন',
    'করলে', 'করলেন', 'করা', 'করাই', 'করায়', 'করার', 'করি', 'করিতে', 'করিয়া', 'করিয়ে', 'করে',
    'করেই', 'করেছিলেন', 'করেছে', 'করেছেন', 'করেন', 'কাউকে', 'কাছ', 'কাছে', 'কাজ', 'কাজে',
    'কারও', 'কারণ', 'কি', 'কিংবা', 'কিছু', 'কিছুই', 'কিন্তু', 'কী', 'কে', 'কেউ', 'কেউই',
    'কেখা', 'কেন', 'কোটি', 'কোন', 'কোনও', 'কোনো', 'ক্ষেত্রে', 'কয়েক', 'খুব', 'গিয়ে', 'গিয়েছে',
    'গিয়ে', 'গুলি', 'গেছে', 'গেল', 'গেলে', 'গোটা', 'চলে', 'চান', 'চায়', 'চার', 'চালু', 'চেয়ে',
    'চেষ্টা', 'ছাড়া', 'ছাড়াও', 'ছিল', 'ছিলেন', 'জন', 'জনকে', 'জনের', 'জন্য', 'জন্যও', 'জে',
    'জানতে', 'জানা', 'জানানো', 'জানায়', 'জানিয়ে', 'জানিয়েছে', 'জে', 'টি', 'ঠিক', 'তখন', 'তত',
    'তথা', 'তবু', 'তবে', 'তা', 'তাঁকে', 'তাঁদের', 'তাঁর', 'তাঁরা', 'তাঁাহারা', 'তাই', 'তাও', 'তাকে',
    'তাতে', 'তাদের', 'তার', 'তারপর', 'তারা', 'তারৈ', 'তাহলে', 'তাহা', 'তাহাতে', 'তাহার', 'তিনঐ',
    'তিনি', 'তিনিও', 'তুমি', 'তুলে', 'তেমন', 'তো', 'তোমার', 'থাকবে', 'থাকবেন', 'থাকা', 'থাকায়',
    'থাকে', 'থাকেন', 'থেকে', 'থেকেই', 'থেকেও', 'দিকে', 'দিতে', 'দিন', 'দিয়ে', 'দিয়েছে', 'দিয়েছেন',
    'দিলেন', 'দু', 'দুই', 'দুটি', 'দুটো', 'দেওয়া', 'দেওয়ার', 'দেওয়া', 'দেখতে', 'দেখা', 'দেখে',
    'দেন', 'দেয়', 'দ্বারা', 'ধরা', 'ধরে', 'ধামার', 'নতুন', 'নয়', 'না', 'নাই', 'নাকি', 'নাগাদ',
    'নানা', 'নিজে', 'নিজেই', 'নিজেদের', 'নিজের', 'নিতে', 'নিয়ে', 'নিয়ে', 'নেই', 'নেওয়া', 'নেওয়ার',
    'নেওয়া', 'নয়', 'পক্ষে', 'পর', 'পরে', 'পরেই', 'পরেও', 'পর্যন্ত', 'পাওয়া', 'পাচ', 'পারি',
    'পারে', 'পারেন', 'পি', 'পেয়ে', 'পেয়্র্', 'প্রতি', 'প্রথম', 'প্রভৃতি', 'প্রযন্ত', 'প্রায়',
    'প্রায়', 'ফলে', 'ফিরে', 'ফের', 'বক্তব্য', 'বদলে', 'বন', 'বরং', 'বলতে', 'বলল', 'বললেন', 'বলা',
    'বলে', 'বলেছেন', 'বলেন', 'বসে', 'বহু', 'বা', 'বাদে', 'বার', 'বি', 'বিনা', 'বিভিন্ন', 'বিশেষ',
    'বিষয়টি', 'বেশ', 'বেশি', 'ব্যবহার', 'ব্যাপারে', 'ভাবে', 'ভাবেই', 'মতো', 'মতোই', 'মধ্যে',
    'মধ্যে', 'মধ্যেই', 'মধ্যেও', 'মনে', 'মাত্র', 'মাধ্যমে', 'মোট', 'মোটেই', 'যখন', 'যত',
    'যতটা', 'যথেষ্ট', 'যদি', 'যদিও', 'যা', 'যাঁর', 'যাঁরা', 'যাওয়া', 'যাওয়ার', 'যাওয়া',
    'যাকে', 'যাচ্ছে', 'যাতে', 'যাদের', 'যান', 'যাবে', 'যায়', 'যার', 'যারা', 'যিনি', 'যে',
    'যেখানে', 'যেতে', 'যেন', 'যেমন', 'র', 'রকম', 'রয়েছে', 'রাখা', 'রেখে', 'লক্ষ', 'শুধু',
    'শুরু', 'সঙ্গে', 'সঙ্গেও', 'সব', 'সবার', 'সমস্ত', 'সম্প্রতি', 'সহ', 'সহিত', 'সাধারণ', 'সামনে',
    'সি', 'সুতরাং', 'সে', 'সেই', 'সেখান', 'সেখানে', 'সেটা', 'সেটাই', 'সেটাও', 'সেটি', 'স্পষ্ট',
    'স্বয়ং', 'হইতে', 'হইবে', 'হইয়া', 'হওয়া', 'হওয়ায়', 'হওয়ার', 'হচ্ছে', 'হত', 'হতে',
    'হতেই', 'হন', 'হবে', 'হবেন', 'হয়', 'হয়তো', 'হয়নি', 'হয়ে', 'হয়েই', 'হয়েছিল', 'হয়েছে',
    'হয়েছেন', 'হল', 'হলে', 'হলেই', 'হলেও', 'হলো', 'হাজার', 'হিসাবে', 'হৈলে', 'হোক', 'হয়',
    'জন্যওজে', 'এসে', 'মধ্যভাগে', 'জ্নজন', 'প্রাথমিক', 'অনুসারে', 'অর্থাৎ',

    'মানে','একটা','সাথে','আসলে','এক','তারপরে','এইটা','মানুষ','ভালো','ছবিতে','হ্যা','হ্যাঁ','হয়ই','হয়ে','হয়তো','যায়','সময়',
    'দেয়','পায়','টাকা','পছন্দ','সুন্দর','বড়','আসে','একটু','কথা','এইরকম','জায়গায়','বাসায়','বুঝতে','চলতে'

    # 'বরুণ','শিউলির','ধাওয়ান','ফ্রেডি','শিউলি','পাই','কাপুর','রনবীর','সারা','ছবি','ইরফান','ফ্রেডির',
    # 'ব্যান্ডের','র‍্যাপসডি'

]


district_tfidf_results = {}

for district, group in test_df.groupby('district'):
    transcriptions = group['transcriptions'].astype(str)

    vectorizer = TfidfVectorizer(
        stop_words=bangla_stopwords,
        token_pattern=r"[\u0980-\u09FF\u200d]{1,}"
    )

    tfidf_matrix = vectorizer.fit_transform(transcriptions)

    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=group['file_name'])

    district_tfidf_results[district] = tfidf_df

district_tfidf_results['tangail']


,অংশ,অই,অইছে,অইতাছিলো,অইতে,অইতেছিলো,অইতো,অইলো,অইয়া,অওয়ার,...,হিস্টরিকাল,হুশ,হেল্প,হো,হোটেল,হোটেলে,হোমোস্যাক্সুয়াল,হোল,হোয়াট,হয়ার
file_name,,,,,,,,,,,,,,,,,,,,,
test_tangail (1).wav,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_tangail (10).wav,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_tangail (100).wav,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.195936,0.000000,0.0,...,0.0,0.0,0.206149,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_tangail (101).wav,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.181319,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_tangail (102).wav,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.146720,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test_tangail (95).wav,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_tangail (96).wav,0.0,0.0,0.133801,0.0,0.0,0.0,0.0,0.179993,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_tangail (97).wav,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
district_tfidf_results['barishal']

,অইছে,অইতাম,অইতে,অইতো,অইবে,অইবেই,অইল,অইলে,অইলো,অইয়া,...,হেয়ারপরও,হোনে,হোনেন,হোম,হোমিপ্যাতিতে,হ্যাগো,হ্যার,হ্যারপর,হ্যারা,হ্যাষে
file_name,,,,,,,,,,,,,,,,,,,,,
test_barishal (1).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.119417,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (10).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.165708,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (100).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (101).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (102).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
test_barishal (95).wav,0.109506,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (96).wav,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.101346,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
test_barishal (97).wav,0.000000,0.0,0.0,0.0,0.256938,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
term_importance = district_tfidf_results['tangail'].sum().sort_values(ascending=False)
term_importance.head(30)

,0
অর,3.707143
নিয়া,3.408356
এইডা,3.204972
অরা,3.114402
অইছে,2.997775
ছিলো,2.951195
পুরা,2.853101
হইছে,2.294116
কইরা,2.228015
ছবিডা,2.144686


In [ ]:
import pandas as pd

def get_top_n_tfidf(district_tfidf_results, top_n=50):
    all_words = []

    for district, tfidf_df in district_tfidf_results.items():
        sorted_tfidf = tfidf_df.sum(axis=0).sort_values(ascending=False)  # Sum TF-IDF per word
        top_n_words = sorted_tfidf.head(top_n)

        # Append to the list with district info
        for word, score in top_n_words.items():
            all_words.append({"word": word, "score": score, "district": district})

    # Create a DataFrame with all the words
    words_df = pd.DataFrame(all_words)

    return words_df

# Get top 50 words for each district in a single dataframe
words_df = get_top_n_tfidf(district_tfidf_results, top_n=50)

# Save each district's top words to a separate CSV file
for district, group_df in words_df.groupby('district'):
    file_name = f"{district}_top_tfidf.csv"
    group_df.to_csv(file_name, index=False)
    print(f"Saved CSV for district: {district}")


Saved CSV for district: barishal
Saved CSV for district: chittagong
Saved CSV for district: habiganj
Saved CSV for district: kishoreganj
Saved CSV for district: narail
Saved CSV for district: narsingdi
Saved CSV for district: rangpur
Saved CSV for district: sandwip
Saved CSV for district: sylhet
Saved CSV for district: tangail


In [ ]:
def get_high_low_tfidf(district_tfidf_results, top_n=10):
    high_tfidf = {}
    low_tfidf = {}

    for district, tfidf_df in district_tfidf_results.items():
        sorted_tfidf = tfidf_df.sum(axis=0).sort_values(ascending=False)  # Sum TF-IDF per word
        high_tfidf[district] = sorted_tfidf.head(top_n).index.tolist()
        low_tfidf[district] = sorted_tfidf.tail(top_n).index.tolist()

    return high_tfidf, low_tfidf


# Get high & low TF-IDF words per district
high_tfidf_words, low_tfidf_words = get_high_low_tfidf(district_tfidf_results)

# Print results for each district
for district, tfidf_df in district_tfidf_results.items():
    print(f"\n District Name: {district}")

    high_words_scores = tfidf_df.sum(axis=0).sort_values(ascending=False).head(10)
    print("\n High TF-IDF Words: ")
    for word, score in high_words_scores.items():
        print(f"  {word}: {score:.4f}")

    low_words_scores = tfidf_df.sum(axis=0).sort_values(ascending=False).tail(10)
    print("\n Low TF-IDF Words: ")
    for word, score in low_words_scores.items():
        print(f"  {word}: {score:.4f}")

    print("-" * 50)



 District Name: barishal

 High TF-IDF Words: 
  এহন: 3.4925
  মোর: 3.4395
  মুই: 3.2446
  কাম: 3.0768
  কয়: 2.9550
  এট্টু: 2.9344
  তয়: 2.7578
  দেহি: 2.6325
  হেই: 2.5265
  লাগে: 2.5243

 Low TF-IDF Words: 
  হেডু: 0.1302
  প্রয়জোন: 0.1272
  মরতে: 0.1272
  ব্যাডারাই: 0.1272
  শরম: 0.1272
  ভাবি: 0.1272
  মরবে: 0.1272
  বুজ্জি: 0.1221
  হেইতো: 0.1221
  সোংসার: 0.1221
--------------------------------------------------

 District Name: chittagong

 High TF-IDF Words: 
  ন: 10.9705
  আঁই: 7.1097
  দে: 7.0657
  আঁর: 6.2915
  অ: 6.2089
  তুই: 6.1568
  ওডি: 5.0751
  দি: 4.9553
  গরি: 4.7210
  এহন: 4.4256

 Low TF-IDF Words: 
  আশতোষ: 0.1514
  বাঁশবাড়িয়া: 0.1514
  বারের: 0.1506
  আসছে: 0.1506
  ভাইও: 0.1443
  আবি: 0.1273
  বানাইয়িদে: 0.1067
  উইটটন: 0.1067
  ইয়ততুন: 0.1067
  আসিয়রে: 0.1067
--------------------------------------------------

 District Name: habiganj

 High TF-IDF Words: 
  বালা: 4.0877
  কিতা: 3.8277
  লাগে: 3.8079
  একটা: 2.8897
  এরফরে: 2.8711
  ইতা: 2.7120
  বাই: 2.6691
